In [1]:
import time
import warnings
from datetime import datetime as dt
from re import M
import ast

import git
import numpy as np
import pandas as pd
from scipy.stats import truncnorm
import scipy.stats as stats

import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', None)

In [20]:
repo = git.Repo(".", search_parent_directories=True).git.rev_parse(
    "--show-toplevel"
)
df = pd.read_csv(f"{repo}/data/cleaning/cleaned_1658850129.csv") 
df_stops = pd.read_excel(
    f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name="MoDstops"
)

df.head(2)

,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two,index
0,5727475e-8224-4302-9228-c92b9d4a5220,f8ff0526-887a-4e48-ad96-977e12fd70c1,5483.0,1.0,4.65,4.65,0.0,False,STANDARD,11009,6004,completed,NaN,2021-06-30 23:12:47,2021-07-01 07:30:00,2021-07-01 07:22:00,637.0,2021-07-01 07:29:37,2021-07-01 07:32:37,2021-07-01 07:25:00,2021-07-01 07:33:55,2021-07-01 07:34:07,2021-07-01 07:33:08,2021-07-01 07:44:27,2021-07-01 07:44:39,2021-07-01 07:44:44,2021-11-03 16:00:44,0.0,457.0,31.0,696.0,1153.0,658.0,495.0,1.06,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18fec0a6-b7ba-442b-8472-04bdb6ba1b86,51e1a1a8-995c-488c-84ce-3789e46f0417,3575.0,1.0,0.00,2.77,0.0,False,BAHN_CARD,2007,3025,canceled,NaN,2021-06-30 22:51:53,2021-07-01 08:50:00,2021-07-01 08:42:00,NaN,NaN,NaN,2021-07-01 08:45:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-01 08:16:03,NaN,NaN,NaN,NaN,NaN,429.0,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_stops = pd.read_excel(
    f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name="MoDstops"
)
df_stops.head(2)

,MoDStop Id,MoDStop Name,MoDStop Lat,MoDStop Long,MoDStop Adresse
0,1001,Mandelgasse,49.351780,8.129000,"Seilerbahn 1, 67433 Neustadt"
1,1002,Hauptfeuerwache,49.353733,8.131552,"Lindenstraße 11, 67433 Neustadt"


In [5]:
df_routes = pd.read_excel(
    f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name="Liste 2022"
)
df_routes.head(2)


,Start #,Start Name,Ende #,Ende Name,Route [m],Luftlinie [m],VRN-eTarif\nohne Bahncard,VRN-eTarif\nmit BahnCard,Qualitätszuschlag,MoD-Fahrpreis \nohne BahnCard,MoD-Fahrpreis\n mit Bahncard,0.12680550240634963,0.07159177456207169,0.18042226487523982,0.5159630940478845,0.2894135567402895,0.72936660268714,price_mod,price_mod_2pers,price_mod_3pers,price_mod_4pers,price_bahncard,price_bahncard_2pers,price_bahncard_3pers,price_bahncard_4pers,price_vrn_surcharge,price_vrn_surcharge_2pers,price_vrn_surcharge_3pers,price_vrn_surcharge_4pers
0,1001,Mandelgasse,1002,Hauptfeuerwache,366,285,1.65,1.24,0.9,2.55,2.14,0.160784,NaN,NaN,0.647059,NaN,NaN,2.55,4.275,6.0,7.725,2.14,3.865,5.18,6.7,0.9,2.625,3.94,5.46
1,1001,Mandelgasse,1003,Kindergarten St. Marien,994,586,1.65,1.24,0.9,2.55,2.14,0.160784,NaN,NaN,0.647059,NaN,NaN,2.55,4.275,6.0,7.725,2.14,3.865,5.18,6.7,0.9,2.625,3.94,5.46


getdistribution()
- help function that returns a probability distribution for continous variables based on mean & standard deviation

In [ ]:
def getdistribution(data, column, min=None, max=None):
    # distribution over scheduled rides
    mean = data[column].median() # use median to better deal with outlier
    std = data[column].std()
    if min==None: 
        a = data[column].min() # min value
    else:
        a = min 
    if max==None:
        b = data[column].max() # max value
    else:
        b = max
    return stats.truncnorm((a - mean) / std, (b - mean) / std, loc=mean, scale=std)

In [6]:
# ## Old but gold 


# ##### TODO: Check in the end if too many rides, which are too short or are not likely enough 
# # Check that no same start and stop address 


# def generateRoute(oldRides, newRides, ridestops, routes):
#     # based on analysis of rides we distinguish between workdays (Monday till Friday noon) and weekend (Friday noon till Sunday)
#     newRideStops = pd.DataFrame(newRides[['scheduled_to', 'pickup_address', 'dropoff_address']], columns=['scheduled_to', 'pickup_address', 'dropoff_address'])
#     newRideStops['day'] = newRideStops['scheduled_to'].apply(lambda x: dt.weekday(x))
#     newRideStops['hour'] = newRideStops['scheduled_to'].apply(lambda x: x.hour)
#     newRideStops['workday'] = np.where(
#         (
#             newRideStops['day'].isin([0,1,2,3,4]) # 0 = Monday, 6 = Sunday
#             & ~(
#                     (newRideStops['day'] == 4) 
#                     & (newRideStops['hour'] > 13)
#             )
#         ),
#         True,
#         False
#     )

#     oldRidestops = pd.DataFrame(oldRides[['state', 'scheduled_to', 'pickup_address', 'dropoff_address']], columns=['state', 'scheduled_to', 'pickup_address', 'dropoff_address'])
#     oldRidestops['scheduled_to'] = pd.to_datetime(oldRidestops['scheduled_to'])
#     oldRidestops['day'] = oldRidestops['scheduled_to'].apply(lambda x: dt.weekday(x))
#     oldRidestops['hour'] = oldRidestops['scheduled_to'].apply(lambda x: x.hour)
#     oldRidestops['workday'] = np.where(
#         (
#             oldRidestops['day'].isin([0,1,2,3,4]) # 0 = Monday, 6 = Sunday
#             & ~(
#                     (oldRidestops['day'] == 4) 
#                     & (oldRidestops['hour'] > 13)
#             )
#         ),
#         True,
#         False
#     )
#     workdayOldRides = oldRidestops[(oldRidestops['workday']==True) & (oldRidestops['state']=='completed')]
#     weekendOldRides = oldRidestops[(oldRidestops['workday']==False) & (oldRidestops['state']=='completed')]

#     # generate ridestops
#     for h in [0] + list(range(7,24)): # rides start between 7:00 and 0:59
#         # timeframe used to get ridestop distribution
#         if h in [23,0]:
#             timeframe = [22,23,0]
#         elif h == 7:
#             timeframe = [7,8,9]
#         else:
#             timeframe = list(range(h-1,h+2))

#         ##### workday ridestop distribution #####
#         # get pickup ridestop distribution of rides on workdays, which are in a +/- 1h timeframe around the planned departure; And add not considered ridestops with minimal frequency count of used stops
#         distPickupWorkday = workdayOldRides[(workdayOldRides['hour'].isin(timeframe))]['pickup_address'].value_counts().rename_axis('pickup_address').reset_index(name='counts')
#         distPickupWorkday = distPickupWorkday.merge(ridestops['MoDStop Id'], left_on='pickup_address', how='outer', right_on='MoDStop Id')
#         distPickupWorkday['pickup_address'] = distPickupWorkday['MoDStop Id']
#         distPickupWorkday = distPickupWorkday.fillna(distPickupWorkday['counts'].min())
#         distPickupWorkday['probabilities'] = (distPickupWorkday.counts / distPickupWorkday.counts.sum())
#         # get dropoff ridestop distribution of rides on workdays, which are in a +/- 1h timeframe around the planned departure; And add not considered ridestops with minimal frequency count of used stops
#         distDropoffWorkday = workdayOldRides[(workdayOldRides['hour'].isin(timeframe))]['dropoff_address'].value_counts().rename_axis('dropoff_address').reset_index(name='counts')
#         distDropoffWorkday = distDropoffWorkday.merge(ridestops['MoDStop Id'], left_on='dropoff_address', how='outer', right_on='MoDStop Id')
#         distDropoffWorkday['dropoff_address'] = distDropoffWorkday['MoDStop Id']
#         distDropoffWorkday = distDropoffWorkday.fillna(distDropoffWorkday['counts'].min())
#         distDropoffWorkday['probabilities'] = (distDropoffWorkday.counts / distDropoffWorkday.counts.sum())

#         ##### weekend ridestop distribution #####
#         # get pickup ridestop distribution of rides on workdays, which are in a +/- 1h timeframe around the planned departure; And add not considered ridestops with minimal frequency count of used stops
#         distPickupWeekend = weekendOldRides[(weekendOldRides['hour'].isin(timeframe))]['pickup_address'].value_counts().rename_axis('pickup_address').reset_index(name='counts')
#         distPickupWeekend = distPickupWeekend.merge(ridestops['MoDStop Id'], left_on='pickup_address', how='outer', right_on='MoDStop Id')
#         distPickupWeekend['pickup_address'] = distPickupWeekend['MoDStop Id']
#         distPickupWeekend = distPickupWeekend.fillna(distPickupWeekend['counts'].min())
#         distPickupWeekend['probabilities'] = (distPickupWeekend.counts / distPickupWeekend.counts.sum())
#         # get dropoff ridestop distribution of rides on workdays, which are in a +/- 1h timeframe around the planned departure; And add not considered ridestops with minimal frequency count of used stops
#         distDropoffWeekend = weekendOldRides[(weekendOldRides['hour'].isin(timeframe))]['dropoff_address'].value_counts().rename_axis('dropoff_address').reset_index(name='counts')
#         distDropoffWeekend = distDropoffWeekend.merge(ridestops['MoDStop Id'], left_on='dropoff_address', how='outer', right_on='MoDStop Id')
#         distDropoffWeekend['dropoff_address'] = distDropoffWeekend['MoDStop Id']
#         distDropoffWeekend = distDropoffWeekend.fillna(distDropoffWeekend['counts'].min())
#         distDropoffWeekend['probabilities'] = (distDropoffWeekend.counts / distDropoffWeekend.counts.sum())

#         # for all new rides planned at time h choose ridestops based on the distributions
#         # pickup_address:
#         newRideStops['pickup_address'] = np.where(
#             (newRideStops['workday'] == True)
#             & (newRideStops['hour'] == h),
#             np.random.choice(distPickupWorkday['pickup_address'], p=distPickupWorkday['probabilities']),
#             np.where(
#                 (newRideStops['workday'] == False)
#                 & (newRideStops['hour'] == h),
#                 np.random.choice(distPickupWeekend['pickup_address'], p=distPickupWeekend['probabilities']),
#                 newRideStops['pickup_address']
#             )
#         )
#         # dropoff_address:
#         newRideStops['dropoff_address'] = np.where(
#             (newRideStops['workday'] == True)
#             & (newRideStops['hour'] == h),
#             np.random.choice(distDropoffWorkday['dropoff_address'], p=distDropoffWorkday['probabilities']),
#             np.where(
#                 (newRideStops['workday'] == False)
#                 & (newRideStops['hour'] == h),
#                 np.random.choice(distDropoffWeekend['dropoff_address'], p=distDropoffWeekend['probabilities']),
#                 newRideStops['dropoff_address']
#             )
#         )

#     # Extract 'distance' and 'shortest_ridetime' based on generated routes
#     newRideStops['distance'] = newRideStops.merge(routes, left_on=['pickup_address', 'dropoff_address'], right_on=['Start #', 'Ende #'], how='left')['Route [m]']
#     newRideStops['shortest_ridetime'] = 1/(30 / (newRideStops['distance'] / 1000) )*60*60 # calculate shortest_ridetime in seconds with average speed of 30 km/h
#     return newRideStops[['pickup_address', 'dropoff_address','distance', 'shortest_ridetime']]



In [7]:
##### TODO: Check in the end if too many rides, which are too short or are not likely enough 


def generateRoute(oldRides, newRides, ridestops, routes):
    # add route identifier to routes dataframe
    allRoutes = routes[routes['Route [m]'] > 500] # Assumption: real rides are at least 500 m long
    allRoutes['route'] = allRoutes['Start #'].astype(str) + "-" + allRoutes['Ende #'].astype(str)

    # based on analysis of rides we distinguish between workdays (Monday till Friday noon) and weekend (Friday noon till Sunday)
    newRideStops = pd.DataFrame(newRides[['created_at', 'scheduled_to', 'pickup_address', 'dropoff_address']], columns=['created_at', 'scheduled_to', 'pickup_address', 'dropoff_address'])
    newRideStops['route'] = ""
    newRideStops['day'] = newRideStops['scheduled_to'].apply(lambda x: dt.weekday(x))
    newRideStops['hour'] = newRideStops['scheduled_to'].apply(lambda x: x.hour)
    newRideStops['workday'] = np.where(
        (
            newRideStops['day'].isin([0,1,2,3,4]) # 0 = Monday, 6 = Sunday
            & ~(
                    (newRideStops['day'] == 4) 
                    & (newRideStops['hour'] > 13)
            )
        ),
        True,
        False
    )

    oldRidestops = pd.DataFrame(oldRides[['state', 'scheduled_to', 'pickup_address', 'dropoff_address']], columns=['state', 'scheduled_to', 'pickup_address', 'dropoff_address'])
    oldRidestops['route'] = oldRidestops['pickup_address'].astype(str) + "-" + oldRidestops['dropoff_address'].astype(str)
    oldRidestops['scheduled_to'] = pd.to_datetime(oldRidestops['scheduled_to'])
    oldRidestops['day'] = oldRidestops['scheduled_to'].apply(lambda x: dt.weekday(x))
    oldRidestops['hour'] = oldRidestops['scheduled_to'].apply(lambda x: x.hour)
    oldRidestops['workday'] = np.where(
        (
            oldRidestops['day'].isin([0,1,2,3,4]) # 0 = Monday, 6 = Sunday
            & ~(
                    (oldRidestops['day'] == 4) 
                    & (oldRidestops['hour'] > 13)
            )
        ),
        True,
        False
    )
    workdayOldRides = oldRidestops[(oldRidestops['workday']==True)]
    weekendOldRides = oldRidestops[(oldRidestops['workday']==False)]

    # generate ridestops
    for h in [0] + list(range(7,24)): # rides start between 7:00 and 0:59
        # timeframe used to get ridestop distribution
        if h in [23,0]:
            timeframe = [22,23,0]
        elif h == 7:
            timeframe = [7,8,9]
        else:
            timeframe = list(range(h-1,h+2))

        ##### workday ridestop distribution #####
        distWorkday = workdayOldRides[(workdayOldRides['hour'].isin(timeframe))]['route'].value_counts().rename_axis('route').reset_index(name='counts')
        numberOfNoise = distWorkday['counts'].sum() / 80 * 20 # 20% noise
        allRoutes['counts'] = distWorkday['counts'].min() # noise is weighted similar to least frequent real driven route
        distWorkday = pd.concat([distWorkday, allRoutes[~allRoutes['route'].isin(distWorkday['route'])].sample(frac=1)[:int(numberOfNoise)][['route', 'counts']]])
        distWorkday['probabilities'] = (distWorkday.counts / distWorkday.counts.sum())

        ##### weekend ridestop distribution #####
        distWeekend = weekendOldRides[(weekendOldRides['hour'].isin(timeframe))]['route'].value_counts().rename_axis('route').reset_index(name='counts')
        numberOfNoise = distWeekend['counts'].sum() / 80 * 20 # 20% noise
        allRoutes['counts'] = distWeekend['counts'].min() # noise is weighted similar to least frequent real driven route
        distWeekend = pd.concat([distWeekend, allRoutes[~allRoutes['route'].isin(distWeekend['route'])].sample(frac=1)[:int(numberOfNoise)][['route', 'counts']]])
        distWeekend['probabilities'] = (distWeekend.counts / distWeekend.counts.sum())

        # split newRideStops dataframe in 1. ride-hour=h & weekend, 2. ride-hour=h & workday, 3. rest
        newRideStops_h_wend = newRideStops[(newRideStops['hour']==h) & (newRideStops['workday']==False)]
        newRideStops_h_work = newRideStops[(newRideStops['hour']==h) & (newRideStops['workday']==True)]
        newRideStops_not_h = newRideStops[~((newRideStops['hour']==h) & (newRideStops['workday']==False)) & ~((newRideStops['hour']==h) & (newRideStops['workday']==True))]

        # generate routes based on distributions
        newRideStops_h_wend['route'] = np.random.choice(distWeekend['route'], p=distWeekend['probabilities'], size=newRideStops_h_wend.shape[0])
        newRideStops_h_work['route'] = np.random.choice(distWorkday['route'], p=distWorkday['probabilities'], size=newRideStops_h_work.shape[0])

        # concat 3 pieces back together
        newRideStops = pd.concat([newRideStops_not_h, newRideStops_h_wend, newRideStops_h_work])

    # Extract pickup & dropoff address from route column
    newRideStops[['pickup_address', 'dropoff_address']] = newRideStops['route'].str.split('-', expand=True)
    newRideStops['pickup_address'] = pd.to_numeric(newRideStops['pickup_address'])
    newRideStops['dropoff_address'] = pd.to_numeric(newRideStops['dropoff_address'])

    # Extract 'distance' and 'shortest_ridetime' based on generated routes
    newRideStops['distance'] = newRideStops.merge(routes, left_on=['pickup_address', 'dropoff_address'], right_on=['Start #', 'Ende #'], how='left')['Route [m]']
    newRideStops['shortest_ridetime'] = 1/(30 / (newRideStops['distance'] / 1000) )*60*60 # calculate shortest_ridetime in seconds with average speed of 30 km/h
    newRideStops.sort_values(by=['created_at'])
    return newRideStops[['pickup_address', 'dropoff_address','distance', 'shortest_ridetime']]



In [8]:
# def generateRoute_simple2(oldRides, newRides, ridestops, routes):
#     oldRideStops = oldRides[['pickup_address', 'dropoff_address']]
#     oldRideStops['route'] = oldRideStops['pickup_address'].astype(str) + "-" + oldRideStops['dropoff_address'].astype(str)
    
#     dist = oldRideStops['route'].value_counts().rename_axis('route').reset_index(name='counts')
#     dist['probabilities'] = (dist.counts / dist.counts.sum())

#     newRideStops = pd.DataFrame(newRides[['pickup_address', 'dropoff_address']], columns=['pickup_address', 'dropoff_address'])
#     newRideStops['route'] = np.random.choice(dist['route'], p=dist['probabilities'], size=newRides.shape[0])
#     newRideStops[['pickup_address', 'dropoff_address']] = newRideStops['route'].str.split('-', expand=True)
#     newRideStops['pickup_address'] = pd.to_numeric(newRideStops['pickup_address'])
#     newRideStops['dropoff_address'] = pd.to_numeric(newRideStops['dropoff_address'])

#     # Extract 'distance' and 'shortest_ridetime' based on generated routes
#     newRideStops['distance'] = newRideStops.merge(routes, left_on=['pickup_address', 'dropoff_address'], right_on=['Start #', 'Ende #'], how='left')['Route [m]']
#     newRideStops['shortest_ridetime'] = 1/(30 / (newRideStops['distance'] / 1000) )*60*60 # calculate shortest_ridetime in seconds with average speed of 30 km/h
    
#     return newRideStops[['pickup_address', 'dropoff_address','distance', 'shortest_ridetime']]

generateCreatedAt()
- function that returns n random 'created_at' timestamps over a period of one specified month based on the probability distribution in original data 
- first step: choose a date from the month based on the probability distribution of rides over the weekdays (Monday-Sunday)
- second step: choose a timestamp based on the probability distribution of rides that are on the same weekday


In [9]:
def generateCreatedAt(oldRides, newRides, m, y):
    # creat list with all days of the month to build up the probability distribution 
    if m == 12:
        m1 = 1
        y1 = y + 1
    else:
        m1 = m + 1
        y1 = y
    daydist = pd.DataFrame(pd.date_range(start=str(m)+'/01/'+str(y), end=str(m1)+'/01/'+str(y1),).to_pydatetime().tolist()[:-1], columns=['date'])
    daydist['weekday'] = daydist['date'].apply(lambda x: dt.weekday(x)) # use the weekday distribution to represent real occurrences of rides

    # extract all dates and their weekday, hour and minute 
    created = pd.DataFrame(pd.to_datetime(df['created_at']), columns=['created_at'])
    created['day'] = created['created_at'].apply(lambda x: dt.weekday(x))
    created['hour'] = created['created_at'].apply(lambda x: x.hour)
    created['minute'] = created['created_at'].apply(lambda x: x.minute)
    
    # get the weekday distribution of old rides
    dist_day = created['day'].value_counts().rename_axis('day').reset_index(name='counts')
    dist_day['probabilities'] = (dist_day.counts / dist_day.counts.sum())
    dist_day = dist_day.sort_values('day')

    # get the hour distribution of old rides per weekday 
    dist_hour = []
    for i in range(0,7):
        dist_hour.append(created[created['day']==i]['hour'].value_counts().rename_axis('hour').reset_index(name='counts'))
        dist_hour[i]['probabilities'] = (dist_hour[i].counts / dist_hour[i].counts.sum())
        dist_hour[i] = dist_hour[i].sort_values('hour')

    
    # get the minute distribution of old rides
    dist_minute = created['minute'].value_counts().rename_axis('minute').reset_index(name='counts')
    dist_minute['probabilities'] = (dist_minute.counts / dist_minute.counts.sum())  
    dist_minute = dist_minute.sort_values('minute')


    # match probability that a ride is on that weekday to all dates in the simulated month
    daydist['probabilities'] =  daydist['weekday'].apply(lambda x: dist_day[dist_day['day']==x]['probabilities'].values[0]) 
    daydist['probabilities'] = daydist['probabilities']/(daydist['probabilities'].sum()) # normalization neccessary to get probability distribution (sum of odds is 1)

    # generate list of values
    values = pd.DataFrame(np.random.choice(daydist['date'], p=daydist['probabilities'], size=newRides.shape[0]), columns=['created_at'])
    values = values.sort_values('created_at')
    values = values.reset_index()
    values['day'] = values['created_at'].apply(lambda x: dt.weekday(x))
    values['created_at'] = values['created_at'] + values['day'].apply(
        lambda x: pd.Timedelta(
            hours=np.random.choice(dist_hour[x]['hour'], p=dist_hour[x]['probabilities']), # choose hour based on distribution of that weekday
            minutes=np.random.choice(dist_minute['minute'], p=dist_minute['probabilities']), # choose minute based on distribution of that hour
            seconds=np.random.choice(list(range(0,60))) # random choice of seconds
        )
    )
    values.sort_values(by=['created_at'])
    return values['created_at']


generateScheduledTo()
- function that returns n random 'scheduled_to' timestamps based on the probability distribution in original data 
- first, consider distribution of scheduled & immediate 
- second, for a scheduled ride add a random prebooking time (based on probability distribution of the prebooking time in original data) to created_at


In [53]:
def generateScheduledTo(oldRides, newRides):
    scheduledNew = pd.DataFrame(columns=['hour'])
    scheduledNew['created_at'] = newRides['created_at']
    scheduledNew['hour'] = scheduledNew['created_at'].apply(lambda x: x.hour)

    # get prebooking time
    scheduled = pd.DataFrame(oldRides[['created_at', 'scheduled_to']], columns=['created_at', 'scheduled_to'])
    scheduled['isScheduled'] = (scheduled.created_at != scheduled.scheduled_to)
    scheduled['created_at'] = pd.to_datetime(scheduled['created_at'])
    scheduled['scheduled_to'] = pd.to_datetime(scheduled['scheduled_to'])
    scheduled['prebook_time'] = scheduled.scheduled_to - scheduled.created_at
    scheduled['prebook_time'] = scheduled['prebook_time'].apply(lambda x: x.total_seconds())
    
    # distribution of prebooked and non-prebooked rides
    dist = scheduled['isScheduled'].value_counts().rename_axis('isScheduled').reset_index(name='counts')
    dist['probabilities'] = (dist.counts / dist.counts.sum())

    # distribution of average prebook time 
    left_border = 8*60 # min value of 8 min -> assumption: scheduled ride must be at least 8 min in the future
    dist_avg_prebook_time = getdistribution(scheduled[scheduled['isScheduled'] == True], 'prebook_time', min=left_border)

    scheduledNew['scheduled_to'] = [(i + pd.Timedelta(dist_avg_prebook_time.rvs(1)[0], unit='seconds')).round(freq='10min') if np.random.choice(dist['isScheduled'], p=dist['probabilities']) else i for i, j in zip(scheduledNew.created_at, scheduledNew.hour)]
    # we have no rides before 7
    scheduledNew['hour'] = scheduledNew['scheduled_to'].apply(lambda x: x.hour)
    scheduledNew['scheduled_to'] = [dt(i.year, i.month, i. day, 7, 0) if j in [1, 2, 3, 4, 5, 6] else i for i, j in zip(scheduledNew.scheduled_to, scheduledNew.hour)]
    return scheduledNew['scheduled_to']

generateDispatchedAt()
- function that returns n random 'dispatched_at' timestamps 
- case 1: scheduled ride -> dispatched_at = scheduled_at - 8 min
- case 2: immediate ride -> dispatched_at = scheduled_at

In [11]:
def generateDispatchedAt(oldRides, newRides):
    scheduled = pd.DataFrame(newRides[['created_at', 'scheduled_to']], columns=['created_at', 'scheduled_to'])
    scheduled['isScheduled'] = (scheduled.created_at != scheduled.scheduled_to)
    scheduled['created_at'] = pd.to_datetime(scheduled['created_at'])
    scheduled['scheduled_to'] = pd.to_datetime(scheduled['scheduled_to'])
    scheduled['dispatched_at'] = np.where(
        scheduled['isScheduled']==True,
        scheduled['scheduled_to'] - pd.Timedelta(minutes=8),
        scheduled['scheduled_to']
    )
    return scheduled['dispatched_at']


generateArrival
- function that returns n random timestamps for 'arriving_push', vehicle_arrived_at', 'pickup_arrival_time'
- first, generate 'vehicle_arrived_at'
    - for scheduled rides: scheduled_to + random scheduling deviation (based on probability distribution of scheduling_deviation=vehicle_arrived_at-scheduled_to in original data)
    - for immediatie rides: scheduled_to + random pickup_arrival_time (based on probability distribution of pickup_arrival_time in original data)
- second, calculate pickup_arrival_time=vehicle_arrived_at-dispatched_at
- third, ..

In [13]:
def generateArrival(oldRides, newRides):
    # get needed information regarding the vehicle arrival in old data
    arrivalOld = pd.DataFrame(oldRides[['created_at', 'scheduled_to', 'dispatched_at', 'vehicle_arrived_at', 'pickup_arrival_time', 'arriving_push']], columns=['created_at', 'scheduled_to', 'dispatched_at', 'vehicle_arrived_at', 'pickup_arrival_time', 'arriving_push'])
    arrivalOld['isScheduled'] = (arrivalOld.created_at != arrivalOld.scheduled_to)
    arrivalOld['created_at'] = pd.to_datetime(arrivalOld['created_at'])
    arrivalOld['scheduled_to'] = pd.to_datetime(arrivalOld['scheduled_to'])
    arrivalOld['vehicle_arrived_at'] = pd.to_datetime(arrivalOld['vehicle_arrived_at'])
    arrivalOld['arriving_push'] = pd.to_datetime(arrivalOld['arriving_push'])

    # create dataframe with needed attributes to determine 'vehicle_arrived_at'
    arrivalNew = pd.DataFrame(newRides[['created_at', 'scheduled_to', 'dispatched_at']], columns=['created_at', 'scheduled_to', 'dispatched_at'])
    arrivalNew['isScheduled'] = (arrivalNew.created_at != arrivalNew.scheduled_to)
    arrivalNew['created_at'] = pd.to_datetime(arrivalNew['created_at'])
    arrivalNew['scheduled_to'] = pd.to_datetime(arrivalNew['scheduled_to'])
    arrivalNew['dispatched_at'] = pd.to_datetime(arrivalNew['dispatched_at'])
    
    ##### generate timestamp 'vehicle_arrived_at' 
    arrivalOld['schedule_deviation'] = arrivalOld.apply(
        lambda row: (
            (row["vehicle_arrived_at"] - row["scheduled_to"] ).round(freq="s")
        ).total_seconds(),
        axis=1,
    )

    # distribution over scheduled rides
    left_border = -8*60 # min value of -8 minutes -> earliest arrived_at
    dist_scheduledRides = getdistribution(arrivalOld[arrivalOld['isScheduled'] == True], 'schedule_deviation', min=left_border )

    # distribution over instant rides - based on pickup_arrival_times distribution
    left_border = 1 # min value of 1 second -> earliest arrived_at
    dist_instantRides = getdistribution(arrivalOld[arrivalOld['isScheduled'] == False], 'pickup_arrival_time', min=left_border )


    # determine timestamp 'vehicle_arrived_at' 
    arrivalNew['vehicle_arrived_at'] = arrivalNew.apply(
        lambda row: (
            (row["scheduled_to"] + pd.Timedelta(dist_scheduledRides.rvs(1)[0], unit='seconds').round(freq="s"))
        )
        if (row["isScheduled"] == True)
        else 
            (row["scheduled_to"] + pd.Timedelta(dist_instantRides.rvs(1)[0], unit='seconds').round(freq="s")),
        axis=1,
    )

    ##### calculate 'pickup_arrival_time'
    arrivalNew["pickup_arrival_time"] = (
        arrivalNew["vehicle_arrived_at"] - arrivalNew["dispatched_at"]
    ).dt.seconds

    ##### generate arriving_push
    # distribution of the time between arriving_push and vehicle_arrived_at
    arrivalOld['deviation_of_arriving_push'] = arrivalOld.apply(
        lambda row: (
            (row["arriving_push"] - row["vehicle_arrived_at"] ).round(freq="s")
        ).total_seconds(),
        axis=1,
    )
    dist = getdistribution(arrivalOld, 'deviation_of_arriving_push')

    # determine timestamp 'arriving_push' 
    arrivalNew['arriving_push'] = arrivalNew.apply(
        lambda row: (
            (row["vehicle_arrived_at"] + pd.Timedelta(dist.rvs(1)[0], unit='seconds').round(freq="s"))
        ),
        axis=1,
    )

    # check that arriving_push is after dispatched_at  
    arrivalNew['arriving_push'] = np.where(
        arrivalNew['dispatched_at'] > arrivalNew['arriving_push'],
        arrivalNew['dispatched_at'] + (arrivalNew['vehicle_arrived_at'] -  arrivalNew['dispatched_at']) * np.random.uniform(0.1,0.9),
        arrivalNew['arriving_push']
    )

    return arrivalNew[['arriving_push', 'vehicle_arrived_at', 'pickup_arrival_time']]


In [14]:
def generatePickup(oldRides, newRides):
    # get needed information regarding the pickup in old data
    pickupOld = pd.DataFrame(oldRides[['created_at', 'scheduled_to', 'dispatched_at', 'vehicle_arrived_at', 'pickup_at', 'pickup_first_eta', 'pickup_eta']], columns=['created_at', 'scheduled_to', 'dispatched_at', 'vehicle_arrived_at', 'pickup_at', 'pickup_first_eta', 'pickup_eta'])
    pickupOld[['created_at', 'scheduled_to', 'dispatched_at', 'vehicle_arrived_at', 'pickup_at', 'pickup_first_eta', 'pickup_eta']] = pickupOld[['created_at', 'scheduled_to', 'dispatched_at', 'vehicle_arrived_at', 'pickup_at', 'pickup_first_eta', 'pickup_eta']].apply(pd.to_datetime)
    pickupOld['isScheduled'] = (pickupOld.created_at != pickupOld.scheduled_to)

    # create dataframe with needed attributes to determine pickup attributes
    pickupNew = pd.DataFrame(newRides[['created_at', 'scheduled_to', 'dispatched_at', 'vehicle_arrived_at']], columns=['created_at', 'scheduled_to', 'dispatched_at', 'vehicle_arrived_at'])
    pickupNew['isScheduled'] = (pickupNew.created_at != pickupNew.scheduled_to)
    pickupNew[['created_at', 'scheduled_to', 'dispatched_at', 'vehicle_arrived_at']] = pickupNew[['created_at', 'scheduled_to', 'dispatched_at', 'vehicle_arrived_at']].apply(pd.to_datetime)
    
    ##### generate earliest_pickup_expectation
    pickupNew['earliest_pickup_expectation'] = pickupNew['dispatched_at'] + pd.Timedelta(minutes=3)

    ##### genrate pickup_at 
    pickupOld['time_until_pickup'] = pickupOld.apply(
        lambda row: (
            (row["pickup_at"] - row["vehicle_arrived_at"] ).round(freq="s")
        ).total_seconds(),
        axis=1,
    )
    # distribution of the time a driver waits until pickup over scheduled rides
    left_border = 1 # min value of 1 second -> earliest arrived_at
    dist_scheduledRides = getdistribution(pickupOld[pickupOld['isScheduled'] == True], 'time_until_pickup', min=left_border )

    # distribution of the time a driver waits until pickup over instant rides
    left_border = 1 # min value of 1 second -> earliest arrived_at
    dist_instantRides = getdistribution(pickupOld[pickupOld['isScheduled'] == False], 'time_until_pickup', min=left_border )

    # determine timestamp 'pickup_at' 
    pickupNew['pickup_at'] = pickupNew.apply(
        lambda row: (
            (row["vehicle_arrived_at"] + pd.Timedelta(dist_scheduledRides.rvs(1)[0], unit='seconds').round(freq="s"))
        )
        if (row["isScheduled"] == True)
        else 
            (row["vehicle_arrived_at"] + pd.Timedelta(dist_instantRides.rvs(1)[0], unit='seconds').round(freq="s")),
        axis=1,
    )

    ##### generate pickup_first_eta
    # distribution of the time between pickup_at and pickup_first_eta
    pickupOld['deviation_of_pickup_first_eta'] = pickupOld.apply(
        lambda row: (
            (row["pickup_first_eta"] - row["pickup_at"] ).round(freq="s")
        ).total_seconds(),
        axis=1,
    )
    dist = getdistribution(pickupOld, 'deviation_of_pickup_first_eta')

    # determine timestamp 'pickup_first_eta' 
    pickupNew['pickup_first_eta'] = pickupNew.apply(
        lambda row: (
            (row["pickup_at"] + pd.Timedelta(dist.rvs(1)[0], unit='seconds').round(freq="s"))
        ),
        axis=1,
    )

    # check that pickup_first_eta is at least 3 min. after created_at 
    pickupNew['pickup_first_eta'] = np.where(
        (pickupNew['created_at'] + pd.Timedelta(minutes=3)) > pickupNew['pickup_first_eta'],
        pickupNew['created_at'] + pd.Timedelta(minutes=3), # created_at + 3 min. is minimum
        pickupNew['pickup_first_eta']
    )
    # check that pickup_first_eta is after dispatched_at 
    pickupNew['pickup_first_eta'] = np.where(
        pickupNew['dispatched_at'] > pickupNew['pickup_first_eta'],
        pickupNew['dispatched_at'] + pd.Timedelta(minutes=3), # TODO: mehr randomness? 
        pickupNew['pickup_first_eta']
    )

    ##### generate pickup_eta
    # distribution of the time between pickup_at and pickup_eta
    pickupOld['deviation_of_pickup_eta'] = pickupOld.apply(
        lambda row: (
            (row["pickup_eta"] - row["pickup_at"] ).round(freq="s")
        ).total_seconds(),
        axis=1,
    )
    dist = getdistribution(pickupOld, 'deviation_of_pickup_eta')

    # determine timestamp 'pickup_eta' 
    pickupNew['pickup_eta'] = pickupNew.apply(
        lambda row: (
            (row["pickup_at"] + pd.Timedelta(dist.rvs(1)[0], unit='seconds').round(freq="s"))
        ),
        axis=1,
    )

    # check that pickup_eta is after dispatched_at  
    pickupNew['pickup_eta'] = np.where(
        pickupNew['dispatched_at'] > pickupNew['pickup_eta'],
        pickupNew['dispatched_at'] + pd.Timedelta(minutes=3), #TODO: mehr randomness
        pickupNew['pickup_eta']
    )

    return pickupNew[['earliest_pickup_expectation', 'pickup_at', 'pickup_eta', 'pickup_first_eta']]

In [80]:
def generateDropoff(oldRides, newRides, routes):
    # get needed information regarding the dropoff in old data
    dropoffOld = pd.DataFrame(oldRides[['pickup_address', 'dropoff_address', 'scheduled_to', 'dropoff_at', 'dropoff_first_eta', 'dropoff_eta', 'ride_time']], columns=['pickup_address', 'dropoff_address', 'scheduled_to', 'dropoff_at', 'dropoff_first_eta', 'dropoff_eta', 'ride_time'])
    dropoffOld[['scheduled_to', 'dropoff_at', 'dropoff_first_eta', 'dropoff_eta']] = dropoffOld[['scheduled_to', 'dropoff_at', 'dropoff_first_eta', 'dropoff_eta']].apply(pd.to_datetime)
    dropoffOld['day'] = dropoffOld['scheduled_to'].apply(lambda x: dt.weekday(x))
    dropoffOld['hour'] = dropoffOld['scheduled_to'].apply(lambda x: x.hour)
    dropoffOld['workday'] = np.where(
        (
            dropoffOld['day'].isin([0,1,2,3,4]) # 0 = Monday, 6 = Sunday
            & ~(
                    (dropoffOld['day'] == 4) 
                    & (dropoffOld['hour'] > 13)
            )
        ),
        True,
        False
    )

    # create dataframe with needed attributes to determine dropoff attributes
    dropoffNew = pd.DataFrame(newRides[['pickup_address', 'dropoff_address', 'scheduled_to', 'pickup_at', 'pickup_first_eta', 'pickup_eta', 'shortest_ridetime']], columns=['pickup_address', 'dropoff_address', 'scheduled_to', 'pickup_at', 'pickup_first_eta', 'pickup_eta', 'shortest_ridetime'])
    dropoffNew[['scheduled_to', 'pickup_at', 'pickup_first_eta', 'pickup_eta']] = dropoffNew[['scheduled_to', 'pickup_at', 'pickup_first_eta', 'pickup_eta']].apply(pd.to_datetime)
    dropoffNew['day'] = dropoffNew['scheduled_to'].apply(lambda x: dt.weekday(x))
    dropoffNew['hour'] = dropoffNew['scheduled_to'].apply(lambda x: x.hour)
    dropoffNew['timeframe'] = dropoffNew['hour'].apply(
        lambda h: (
            [22,23,0]
            if h in [23,0]
            else 
            ([7,8,9]
            if h == 7
            else
            list(range(h-1,h+2))) 
            )
    )
    dropoffNew['workday'] = np.where(
        (
            dropoffNew['day'].isin([0,1,2,3,4]) # 0 = Monday, 6 = Sunday
            & ~(
                    (dropoffNew['day'] == 4) 
                    & (dropoffNew['hour'] > 13)
            )
        ),
        True,
        False
    )

    ##### generate ride_time based on ride_time of most similar rides
    dropoffNew['ride_time'] = dropoffNew.apply(
        lambda row:
            # if rides exist with same route & workday/weekend flag & in a timeframe of +/-1 hour
            round(dropoffOld[(dropoffOld['pickup_address']==row['pickup_address']) 
                & (dropoffOld['dropoff_address']==row['dropoff_address'])
                & (dropoffOld['workday']==row['workday'])          
                & (dropoffOld['hour'].isin(row['timeframe']))]['ride_time'].mean())
            if len(dropoffOld[(dropoffOld['pickup_address']==row['pickup_address']) 
                & (dropoffOld['dropoff_address']==row['dropoff_address'])
                & (dropoffOld['workday']==row['workday'])                   
                & (dropoffOld['hour'].isin(row['timeframe']))]['ride_time']) > 0
            else
            # if rides exist with same route & in a timeframe of +/-1 hour - workday/weekend does not matter
            round(dropoffOld[(dropoffOld['pickup_address']==row['pickup_address']) 
                & (dropoffOld['dropoff_address']==row['dropoff_address'])
                & (dropoffOld['hour'].isin(row['timeframe']))]['ride_time'].mean())
            if len(dropoffOld[(dropoffOld['pickup_address']==row['pickup_address']) 
                & (dropoffOld['dropoff_address']==row['dropoff_address'])
                & (dropoffOld['hour'].isin(row['timeframe']))]['ride_time']) > 0
            else
            # if rides exist with same route - day & hour does not matter
            round(dropoffOld[(dropoffOld['pickup_address']==row['pickup_address']) 
                & (dropoffOld['dropoff_address']==row['dropoff_address'])]['ride_time'].mean())
            if len(dropoffOld[(dropoffOld['pickup_address']==row['pickup_address']) 
                & (dropoffOld['dropoff_address']==row['dropoff_address'])]['ride_time']) > 0
            else
            # else, use shortest ridetime: 30km/h over distance of the route
            round((routes[(routes['Start #']==row['pickup_address']) 
                & (routes['Ende #']==row['dropoff_address'])]['Route [m]'].values[0] * 3600 / 30000) * np.random.uniform(1.0,1.2)),
            axis=1,
    )

    ##### genereate dropoff_at 
    dropoffNew['dropoff_at'] = dropoffNew['pickup_at'] + pd.to_timedelta(dropoffNew['ride_time'], unit='seconds')

    ##### generate dropoff_first_eta
    dropoffNew['dropoff_first_eta'] = dropoffNew['pickup_first_eta'] + pd.to_timedelta(dropoffNew['shortest_ridetime'], unit='seconds')

    ##### generate dropoff_eta
    # distribution of the time between dropoff_at and dropoff_eta
    dropoffOld['deviation_of_dropoff_eta'] = dropoffOld.apply(
        lambda row: (
            (row["dropoff_eta"] - row["dropoff_at"] ).round(freq="s")
        ).total_seconds(),
        axis=1,
    )
    dist = getdistribution(dropoffOld, 'deviation_of_dropoff_eta')

    # determine timestamp 'dropoff_eta' 
    dropoffNew['dropoff_eta'] = dropoffNew.apply(
        lambda row: (
            (row["dropoff_at"] + pd.Timedelta(dist.rvs(1)[0], unit='seconds').round(freq="s"))
        ),
        axis=1,
    )

    # check that dropoff_eta is after pickup_eta & pickup_at
    dropoffNew['dropoff_eta'] = np.where(
        (dropoffNew['pickup_eta'] > dropoffNew['dropoff_eta'])
        | (dropoffNew['pickup_at'] > dropoffNew['dropoff_eta']),
        dropoffNew['dropoff_at'] + pd.Timedelta(minutes=3), # TODO: mehr randomness
        dropoffNew['dropoff_eta']
    )


    return dropoffNew[['dropoff_at', 'dropoff_eta', 'dropoff_first_eta']]
    

generateValues()
- general function that returns n random values based on the probability distribution of a certain column
- used for the following ride attributes: number_of_passenger, free_ride, payment_type, arrival_indicator, rating


In [16]:
def generateValues(column_name, df, newRides):
    dist = df[column_name].value_counts().rename_axis(column_name).reset_index(name='counts')
    dist['probabilities'] = (dist.counts / dist.counts.sum())
    return np.random.choice(dist[column_name], p=dist['probabilities'], size=newRides.shape[0])


In [17]:
# Attributes: ['pickup_arrival_time', 'arrival_deviation', 'waiting_time', 'boarding_time', 'ride_time', 'trip_time', 'shortest_ridetime', 'delay', 'longer_route_factor']
def generateTimeperiods(newRides):
    # Attribute: 'arrival_deviation'
    newRides["arrival_deviation"] = newRides.apply(
        lambda row: (
            (row["vehicle_arrived_at"] - row["arriving_push"]).round(freq="s")
        ).total_seconds()
        - 180
        if (row["vehicle_arrived_at"] == row["vehicle_arrived_at"])
        and (row["arriving_push"] == row["arriving_push"])
        else np.NaN,
        axis=1,
    )

    # Attribute: 'waiting_time'
    newRides["waiting_time"] = newRides.apply(
        lambda row: (
            (row["vehicle_arrived_at"] - row["earliest_pickup_expectation"]).round(
                freq="s"
            )
        ).total_seconds()
        if (row["vehicle_arrived_at"] == row["vehicle_arrived_at"])
        and (row["earliest_pickup_expectation"] == row["earliest_pickup_expectation"])
        else np.NaN,
        axis=1,
    )

    # Attribute: 'boarding_time'
    newRides["boarding_time"] = newRides.apply(
        lambda row: (
            (row["pickup_at"] - row["vehicle_arrived_at"]).round(freq="s")
        ).total_seconds()
        if (row["vehicle_arrived_at"] == row["vehicle_arrived_at"])
        and (row["pickup_at"] == row["pickup_at"])
        else np.NaN,
        axis=1,
    )

    # Attribute: 'ride_time'
    newRides["ride_time"] = newRides.apply(
        lambda row: (
            (row["dropoff_at"] - row["pickup_at"]).round(freq="s")
        ).total_seconds()
        if (row["dropoff_at"] == row["dropoff_at"])
        and (row["pickup_at"] == row["pickup_at"])
        else np.NaN,
        axis=1,
    )

    # Attribute: 'trip_time'
    newRides["trip_time"] = newRides.apply(
        lambda row: (row["ride_time"] + row["waiting_time"]),
        axis=1,
    )

    # Attribute: 'delay'
    newRides["delay"] = newRides.apply(
        lambda row: (row["trip_time"] - row["shortest_ridetime"]),
        axis=1,
    )

    # Attribute: 'longer_route_factor'
    newRides["longer_route_factor"] = newRides.apply(
        lambda row: round(row["ride_time"] / row["shortest_ridetime"], 2)
        if (row["shortest_ridetime"] != 0)
        else np.NaN,
        axis=1,
    )

    return newRides[['arrival_deviation', 'waiting_time', 'boarding_time', 'ride_time', 'trip_time', 'delay', 'longer_route_factor']] 

In [35]:
def generateRideSpecs(oldRides,ridestops, routes, n, month, year):
    timestamp = str(round(time.time()))
    newRides = pd.DataFrame(columns=oldRides.columns)
    oldRides = oldRides[oldRides['state']=='completed']
    newRides['id'] = [timestamp + '-' + str(x) for x in list(range(0,n))]
    newRides['user_id'] = [str(x) + '-' + timestamp for x in list(range(0,n))] # Ein Kunde mehrere Rides
    newRides['number_of_passenger'] = generateValues('number_of_passenger', oldRides, newRides)
    newRides['free_ride'] = generateValues('free_ride', oldRides, newRides)
    newRides['payment_type'] = generateValues('payment_type', oldRides, newRides)
    newRides['state'] = 'completed'
    newRides['arrival_indicator'] = generateValues('arrival_indicator', oldRides, newRides)
    newRides['rating'] = generateValues('rating', oldRides, newRides) #zufällig ratings rein, die nicht bisher gerated wurden? Oder Rating ganz raus?
    newRides['created_at'] = generateCreatedAt(oldRides, newRides, month, year)
    newRides['scheduled_to'] = generateScheduledTo(oldRides, newRides)
    newRides[['pickup_address', 'dropoff_address','distance', 'shortest_ridetime']] = generateRoute(oldRides, newRides, ridestops, routes) # prices are not considered
    # newRides[['pickup_address', 'dropoff_address','distance', 'shortest_ridetime']] = generateRoute_simple(oldRides, newRides, ridestops, routes) # prices are not considered
    # newRides[['pickup_address', 'dropoff_address','distance', 'shortest_ridetime']] = generateRoute_simple2(oldRides, newRides, ridestops, routes) # prices are not considered
    newRides['dispatched_at'] = generateDispatchedAt(oldRides, newRides)
    newRides[['arriving_push','vehicle_arrived_at', 'pickup_arrival_time']] = generateArrival(oldRides, newRides)
    newRides[['earliest_pickup_expectation', 'pickup_at', 'pickup_eta', 'pickup_first_eta']] = generatePickup(oldRides, newRides)
    newRides[['dropoff_at', 'dropoff_eta', 'dropoff_first_eta']] = generateDropoff(oldRides, newRides, routes)
    newRides[['arrival_deviation', 'waiting_time', 'boarding_time', 'ride_time', 'trip_time', 'delay', 'longer_route_factor']] = generateTimeperiods(newRides)
    return newRides


In [ ]:
newRides = pd.DataFrame(columns=df.columns)
warnings.filterwarnings('ignore')
newRides = generateRideSpecs(df, df_stops, df_routes, 9002, 6, 2022)
newRides[['arriving_push', 'vehicle_arrived_at', 'created_at', 'scheduled_to', 'dispatched_at','vehicle_arrived_at', 'pickup_arrival_time', 'dropoff_at', 'dropoff_eta', 'pickup_first_eta', 'shortest_ridetime']].head(10)


# TODO: arrived_at darf nicht vor dispatched_at liegen 


In [44]:
newRides.to_excel("simulated.xlsx")

In [45]:
df_completed = df[df['state']=='completed']

In [ ]:
newRides[newRides['earliest_pickup_expectation'] > newRides['dropoff_at']]

In [127]:
len(df_completed[df_completed['earliest_pickup_expectation'] > df_completed['dropoff_at']])

62

In [90]:
df_completed[['arriving_push', 'vehicle_arrived_at', 'created_at', 'scheduled_to', 'pickup_at', 'pickup_first_eta', 'pickup_eta']] = df_completed[['arriving_push', 'vehicle_arrived_at', 'created_at', 'scheduled_to', 'pickup_at', 'pickup_first_eta', 'pickup_eta']].apply(pd.to_datetime)

In [ ]:
df_completed[(df_completed['scheduled_to'] > df_completed['pickup_eta'] ) ]#& (df_completed['dropoff_address'] == 3012)]

In [ ]:

df_completed['new']

In [132]:
pd.to_datetime(df['vehicle_arrived_at']).dt.day

0         1.0
1         NaN
2         1.0
3         NaN
4         NaN
         ... 
18957    31.0
18958     NaN
18959     5.0
18960     2.0
18961    20.0
Name: vehicle_arrived_at, Length: 18961, dtype: float64